In [1]:
%ls
import matplotlib.pyplot as plt
from matplotlib import rc
from matplotlib import cm
from matplotlib import gridspec
import numpy as np
import pandas as pd
import subprocess, collections
import glob, tqdm, os
rc('font', weight='bold')

plot_ee_trace_bias.ipynb  qsub.sh  REF_L/  REF_RL/


In [7]:

RL_project, L_project = 'REF_RL', 'REF_L'
last_nanoseconds = 25
cutoff = 10

if not os.path.exists('plots'):
    os.makedirs('plots')

for project in [RL_project,L_project]:
    if project == RL_project:
        project_type = 'RL'
        swaps_per_ns = 2000 # 4fs :: 1000 : 2fs
        lambdas = 99
        fe_ndx = 6
    elif project == L_project:
        project_type = 'L'
        swaps_per_ns = 2000 # 4fs
        lambdas = 99
        fe_ndx = 6
        
    prefix = '.' #CLONE0' #f'/home/yabmtm/Downloads/RL_logs' #f'p{project}'# I use this so it's more portable
    
    for x in range(3): #range(len(glob.glob(f'{prefix}/RUN*'))):
        run = f'RUN{x}'
        print(f'Processing {run}')
        if not os.path.exists(f'{prefix}/{project}/{run}/md.log'):
            print(f'Missing {prefix}/{project}/{run}/md.log')
            continue
        increments, lambda_indices, frame_energies, free_energies, matrices = [],[],[],[],[]
        read_energies, read_matrix = False, False
        for line in tqdm.tqdm(open(f'{prefix}/{project}/{run}/md.log')):
            if 'Wang-Landau incrementor is:' in line:
                increments.append(float(line.split()[-1]))
            elif '<<' in line:
                lambda_indices.append(int(line.split()[0]))
            if f'1  0.000  0.000  0.000' in line:
                read_energies = True
            if read_energies:
                try:
                    frame_energies.append(float(line.split()[fe_ndx]))
                except Exception as e:
                    print(line.split())
            if len(frame_energies) == lambdas:
                free_energies.append(frame_energies)
                frame_energies, read_energies = [],False
            if '                     Transition Matrix' in line:
                read_matrix = True
                matrices.append([])
            if read_matrix:
                matrices[-1].append(line.split()[:-1])
                if len(matrices[-1]) == lambdas:
                    matrices[-1] = [[float(x)*100 for x in line] for line in matrices[-1][2:]]
                    read_matrix = False

        free_energies = np.asarray(free_energies)
        # ndx = int(z*swaps_per_ns)
        try:
            print(np.shape(free_energies))
            last_energies = free_energies[len(free_energies)-int(
              swaps_per_ns * last_nanoseconds):][:,-1]
        except Exception as e:
            print(e)
            continue
        end_variance = max(last_energies) - min(last_energies)
        print(run, end_variance)
        if 1: #end_variance < cutoff:
          #  try:
                np.save(f'{prefix}/{project}_RUN{x}_feb.npy', free_energies[::10])
                np.save(f'{prefix}/{project}_RUN{x}_lam.npy', lambda_indices[::10])
                np.save(f'{prefix}/{project}_RUN{x}_inc.npy', increments[::10])

            # plot lambda index over time colored by bias magnitude
                plt.figure(figsize=(18,10))
                gs = gridspec.GridSpec(2, 2, width_ratios=[5,2]) 
                lambda_indices = [int(x) for x in lambda_indices]
                occurrences = collections.Counter(lambda_indices)
                keys = list(occurrences.keys())
                occurrences = [occurrences[x] for x in occurrences]
                min_sampled_lam = keys[occurrences.index(min(occurrences))]
                weights = list(reversed(sorted(set(increments))))
                colors = cm.rainbow(np.linspace(1,0,len(weights)))
                scatter_colors = [colors[weights.index(x)] for x in increments]
                ax2 = plt.subplot(gs[0])
                ax2.scatter(range(len(free_energies)), free_energies[:,-1], s=6, c=scatter_colors)
                ax2.set_ylim(free_energies[:,-1][-1]-20,free_energies[:,-1][-1]+20)
                ax3 = plt.subplot(gs[1])
                ax3.matshow(matrices[-1], cmap=plt.cm.viridis)
                ax0 = plt.subplot(gs[2])
                ax0.scatter(range(len(lambda_indices)), lambda_indices, s=6, c=scatter_colors)
                #print(min_sampled_lam,min(occurrences))
                ax2.axhline(y=np.average(free_energies[:,-1])) #min_sampled_lam)
                ax0.axhline(y=min_sampled_lam)
                ax0.set_xlabel('Frame Index')
                ax0.set_ylabel('Lambda Index')
                ax0.set_title(f'Lambda Trace for {run}')

                # plot weighted histogram of lambda indexes colored by bias magnitude
                ax1 = plt.subplot(gs[3])
                barWidth = 1
                increments = [float(x) for x in increments]
                cumulative_heights = [0 for x in range(lambdas)]
                for z,i in enumerate(reversed(sorted(set(increments)))):
                    wl_increment_indices = [x for x in range(len(increments)) if increments[x] == i]
                    lambda_indices_inc = [lambda_indices[x] for x in wl_increment_indices]
                    occurrences = collections.Counter(lambda_indices_inc)
                    heights = [occurrences[x]*i for x in range(1,lambdas+1)]
                    try:
                        ax1.bar(range(1,lambdas+1), heights, bottom=cumulative_heights, color=colors[z], width=barWidth)
                    except Exception as e:
                        print(e, len(heights), len(colors))
                        continue
                    cumulative_heights = [cumulative_heights[x] + heights[x] for x in range(lambdas)]
                ax1.set_xlabel('Lambda Index')
                ax1.set_ylabel('Cumulative Bias (kT)')
                ax1.set_title(f'Weighted Histogram of Sampling for {run}')
                plt.tight_layout()
                plt.savefig(f'plots/{project}_{run}.png')
                print('DONE')
                plt.close()
          #  except Exception as e:
          #      print('p{project}/RUN{run} Failed to plot.')
          #      continue

Processing RUN0


16580373it [00:22, 723616.90it/s]


(139083, 99)
RUN0 14.995130000000017
DONE
Processing RUN1


16848887it [00:23, 715500.50it/s]


(141336, 99)
RUN1 11.844259999999991
DONE
Processing RUN2


...

(26544, 99)
RUN0 11.452910000000003
DONE
Processing RUN1


3468000it [00:05, 607545.90it/s]


(28695, 99)
RUN1 3.3804599999999994
DONE
Processing RUN2


3816057it [00:06, 587130.72it/s]


(31521, 99)
RUN2 9.897140000000007
DONE
